In this notebook we compare direct simulation of stegotons with pseudospectral solution of the homogenized equations.

In [ ]:
from clawpack import pyclaw
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation
import numpy as np
from stegoton import *
from IPython.display import HTML
from scipy.integrate import quad

font = {'size'   : 15}
matplotlib.rc('font', **font)
ifft = np.fft.ifft

In [ ]:
def stress(frame):
    """Compute stress from strain and momentum"""
    from clawpack.riemann.nonlinear_elasticity_1D_py import sigma 
    aux = frame.aux
    epsilon = frame.q[0,:]
    stress = sigma(frame.q,aux,{'stress_relation':'exponential'})
    return stress

In [ ]:
tfinal = 100.
A = 0.5

# DNS via Clawpack

In [ ]:
claw = setup(ic=2, num_output_times=100, tfinal=tfinal, cells_per_layer=40, amp=A)
claw.run()

In [ ]:
q = claw.frames[-1].q
xc = claw.frames[0].grid.x.centers
rho = claw.frames[0].aux[0,:]
u = q[1,:]/rho
sigma = stress(claw.frames[-1])
plt.plot(xc,sigma)
#plt.xlim(50,100);

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.02,0.25)
ax[1].set_ylim(-0.02,1.4)
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    tx = 1.*t + 20
    xl = max(tx-60,0)
    xr = max(tx, 60)

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('-u',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    ax[0].set_xlim(xl,xr)
    ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=101, repeat=False)
plt.close()
HTML(anim.to_jshtml())

# Homogenized equation solution with pseudospectral method

Here we solve the homogenized equations.  We have specialized to a piecewise-constant medium with exponential stress-strain relation
$$
    \sigma(\epsilon,x) = \exp(K(x)\epsilon)-1.
$$

The homogenized equations are then
\begin{align}
    \sigma_t & = -\frac{1}{\left<K^{-1}\right>} \left(-(\sigma+1)u_x + \delta C_{11}u_{xx} + \delta^2 C_{12} (\sigma+1) u_{xxx} + \delta^2 C_{13} \sigma_x u_{xx}\right) \\
    u_t & = -\frac{1}{\left<\rho\right>} \left( -\sigma_x + \delta C_{21} 
    \sigma_{xx} +\delta^2 C_{22} \sigma_{xxx}\right)
\end{align}

## Averaged coefficients: piecewise-constant media

You should modify `setaux()` in `stegoton.py` to generate the appropriate medium and then run either this section or the next one (not both).

In [ ]:
ZA = np.sqrt(rhoA*KA)
ZB = np.sqrt(rhoB*KB)
cA = np.sqrt(KA/rhoA)
cB = np.sqrt(KB/rhoB)

rho_mean = (rhoA+rhoB)/2
Kinv_mean = (1/KA+1/KB)/2

In [ ]:
# Homogenized PDE coefficients
alpha_fac = -1/12 * alpha**2 * (1-alpha)**2
C12 = alpha_fac * (rhoA-rhoB)*(ZA**2-ZB**2) / (KA*KB*Kinv_mean*rho_mean**2)
C22 = alpha_fac * (KA-KB)*(ZA**2-ZB**2) / (KA**2*KB**2*Kinv_mean**2*rho_mean)
C13 = alpha_fac * ( rho_mean**2*(KA-KB)**2 + \
                   (ZA**2-ZB**2)**2) / (KA**2*KB**2*Kinv_mean**2*rho_mean**2)

C11 = 0.
C21 = 0.

## Averaged coefficients: general case

In [ ]:
eps = 1.e-4

def mean(f):
    return quad(f,0,1,epsabs=eps,epsrel=eps)[0]

def bracket(f):
    brace = lambda y: f(y)-mean(f)
    brack_nzm = lambda y: quad(brace,0,y,epsabs=eps,epsrel=eps)[0]
    mean_bracket = mean(brack_nzm)
    def brack(y):
        return quad(brace,0,y,epsabs=eps,epsrel=eps)[0] - mean_bracket
    return brack

In [ ]:
# These functions should be modified if necessary to match setaux() in stegoton.py:
rho = lambda x: 4+3.5*np.sin(2*np.pi*x)
K = lambda x: 4+3.5*np.cos(2*np.pi*x)

#K = lambda x: np.exp(np.cos(2*np.pi*x))
#def K(x):
#    xfrac = x-np.floor(x)
#    return 0.5*(xfrac<0.5) + 7.5*(xfrac>=0.5)

rho_mean = mean(rho)
Kinv = lambda x: 1./K(x)
Kinv_mean = mean(Kinv)
brho = bracket(rho)
bKinv = bracket(Kinv)

In [ ]:
numer1 = lambda x: brho(x)/K(x)
C11 = mean(numer1)/(rho_mean*Kinv_mean)
numer2 = lambda x: rho(x)*bKinv(x)
C21 = mean(numer2)/(rho_mean*Kinv_mean)

In [ ]:
numerA = lambda x: bKinv(x)*brho(x)
numerB = lambda x: Kinv(x)*(brho(x))**2
numerC = lambda x: rho(x)*(bKinv(x))**2

C12 = mean(numerA)/(rho_mean*Kinv_mean) - mean(numerB)/(rho_mean**2*Kinv_mean)
C22 = mean(numerA)/(rho_mean*Kinv_mean) - mean(numerC)/(rho_mean*Kinv_mean**2)
C13 = 2*mean(numer2)**2/(rho_mean*Kinv_mean)**2 + 2*mean(numerA)/(rho_mean*Kinv_mean) \
   - 2*mean(numerC)/(rho_mean*Kinv_mean**2) - mean(numerB)/(rho_mean**2*Kinv_mean)

In [ ]:
C11

In [ ]:
C21

In [ ]:
C13

In [ ]:
C22

## Pseudospectral solution

In [ ]:
delta = 1.

def rk3(u,xi,rhs,dt,du):
    y2 = u + dt*rhs(u,du,xi)
    y3 = 0.75*u + 0.25*(y2 + dt*rhs(y2,du,xi))
    u_new = 1./3 * u + 2./3 * (y3 + dt*rhs(y3,du,xi))
    return u_new

def rhs(q, dq, xi):
    sigma = q[0,:]
    u = q[1,:]
    sigmahat = np.fft.fft(sigma)
    uhat = np.fft.fft(u)
    
    sigma_x =   np.real(ifft(1j*xi*sigmahat))
    sigma_xx = np.real(ifft(-   xi**2*sigmahat))
    sigma_xxx = np.real(ifft(-1j*xi**3*sigmahat))
    u_x =       np.real(ifft(1j*xi*uhat))
    u_xx =      np.real(ifft(-  xi**2*uhat))
    u_xxx =     np.real(ifft(-1j*xi**3*uhat))
    
    sp1 = sigma+1
    
    dsigma = -1./Kinv_mean * (-sp1*u_x + 1*C11*sp1*u_xx + C12*sp1*u_xxx + C13*sigma_x*u_xx)
    du = -1./rho_mean * (-sigma_x + 1*C21*sigma_xx + C22*sigma_xxx)
    
    dq[0,:] = dsigma
    dq[1,:] = du
    return dq

def solve_NLE(width=10.0, L=200, tfinal=100., m=256):
    """Solve the homogenized nonlinear elasticity equations using 
       Fourier spectral collocation in space
       and SSPRK3 in time, on the domain (-L/2,L/2).
    """
    # Grid
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    dt = 1.73/((m/2)**2) 
    dt = 1*dt# / np.abs(C21/rhoave)  # Coefficient of a dispersive term
    print(dt)
    
    sigma0 = A * np.exp(-x**2 / width**2)
    u0 = np.zeros_like(x)
    q = np.zeros((2,len(x)))
    q[0,:] = sigma0
    q[1,:] = u0

    dq = np.zeros_like(q)

    num_plots = 100
    nplt = np.floor((tfinal/num_plots)/dt)
    nmax = int(round(tfinal/dt))
    print(nmax, nmax*dt)

    fig = plt.figure(figsize=(12,8))
    axes = fig.add_subplot(111)
    line, = axes.plot(x,q[0,:],lw=2)
    xi_max = np.max(np.abs(xi))
    axes.set_xlabel(r'$x$',fontsize=30)
    plt.close()

    sigma = [q[0,:].copy()]
    u = [q[1,:].copy()]
    tt = [0]
    
    for n in range(1,nmax+1):
        q_new = rk3(q,xi,rhs,dt,dq)
            
        q = q_new.copy()
        t = n*dt
        #print(t)
        # Plotting
        if np.mod(n,nplt) == 0:
            print(n//nplt)
            sigma.append(q[0,:].copy())
            u.append(q[1,:].copy())
            tt.append(t)
        
    def plot_frame(i):
        line.set_data(x,sigma[i])
        axes.set_title('t= %.2e' % tt[i])
        #axes.set_xlim((-np.pi,np.pi))
        #axes.set_ylim(ylims)

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                       frames=101, interval=100)

    return HTML(anim.to_jshtml()), sigma, u, x

In [ ]:
anim, sigma_hom, u_hom, x_hom = solve_NLE(m=256, width=3, tfinal=tfinal)

In [ ]:
anim

In [ ]:
anim

In [ ]:
plt.plot(x_hom,sigma_hom[0])
plt.plot(xc,stress(claw.frames[0]))

In [ ]:
i = 90
plt.figure(figsize=(12,8))
plt.plot(x_hom,sigma_hom[i])
plt.plot(xc,stress(claw.frames[i]))
#plt.xlim(50,100)
plt.legend(['Homogenized','Direct'])

# Conclusions

- For media in which the $O(\delta)$ terms vanish (such as piecewise-constant media), the agreement is still fairly good when the solitary waves first separate.
- The agreement gets worse as you get closer to the wave-breaking regime.
- For media in which the $O(\delta)$ terms don't vanish, it seems that those terms are insufficient to prevent wave breaking?  We find that the $O(\delta^2)$ terms are still essential even at quite short times.  Omitting the $O(\delta)$ terms seems to give better agreement than if we include them!  Suggests that they are not implemented correctly, but also that they are insignificant.
- For some reason, the pseudospectral discretization is stable with $\Delta t = O(\Delta x^2)$!